In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 이전 대화 내용을 기억하는 multi-turn Chain

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder #추가
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory #추가
#from langchain_community.chat_message_histories import SQLChatMessageHistory - SQL에 저장할때 사용
from langchain_core.runnables.history import RunnableWithMessageHistory #추가

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful Q&A assistant.'),
        MessagesPlaceholder(variable_name="chat_history"),
        ('human', 
         '#Question:\n{question} \n#Context\n{context}')
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")

In [17]:
chain = prompt | llm | StrOutputParser()

In [18]:
# 대화를 기록하는 체인 chain_with_history

In [19]:
store = {}

# 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()

    return store[session_ids]

In [20]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history, # 세션 기록을 가져오는 함수
    input_messages_key="question", # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history" #기록 메세지의 키
)

In [22]:
chain_with_history.invoke({"question": "나의 이름은 재호야. 내가 결혼할 사람은 진원이야"},
                           config={"configurable": {"session_id":"abc123"}})

[대화 세션ID]: abc123


'안녕하세요, 재호님! 결혼할 사람은 진원이군요. 축하드립니다! 두 분의 행복한 미래를 기원합니다. 결혼 준비는 잘 진행되고 있나요?'

In [23]:
chain_with_history.invoke({"question": "내가 누구랑 결혼한다고?"},
                           config={"configurable": {"session_id":"abc123"}})

[대화 세션ID]: abc123


'당신은 진원이와 결혼한다고 하셨습니다. 결혼 준비는 잘 되고 있나요?'

In [24]:
chain_with_history.invoke({"question": "내가 누구랑 결혼한다고?"},
                           config={"configurable": {"session_id":"abc1234"}})

[대화 세션ID]: abc1234


'결혼 상대에 대한 정보는 개인적인 사안이기 때문에 제가 알 수 없습니다. 결혼 상대에 대해 고민하고 있다면, 그 사람과의 관계, 가치관, 미래 계획 등을 고려해보는 것이 좋습니다. 도움이 필요하다면 구체적인 질문을 해주시면 좋겠습니다!'

: 